# Лекция 8. Работа с базами данных 

Использование баз данных - это еще один способ хранения информации. Базы данных полезны не только в хранении информации. Используя СУБД, можно делать срезы информации по различным параметрам.

**База данных (БД)** - это данные, которые хранятся в соответствии с определенной схемой. В этой схеме каким-то образом описаны соотношения между данными.

**Язык БД (лингвистические средства)** - используется для описания структуры БД, управления данными (добавление, изменение, удаление, получение), управления правами доступа к БД и ее объектам, управления транзакциями.

**Система управления базами данных (СУБД)** - это программные средства, которые дают возможность управлять БД. СУБД должны поддерживать соответствующий язык (языки) для управления БД.

# SQL

**SQL (structured query language)** - используется для описания структуры БД, управления данными (добавление, изменение, удаление, получение), управления правами доступа к БД и ее объектам, управления транзакциями.

Язык SQL подразделяется на такие категории:

* DDL (Data Definition Language) - язык описания данных
* DML (Data Manipulation Language) - язык манипулирования данными
* DCL (Data Control Language) - язык определения доступа к данным
* TCL (Transaction Control Language) - язык управления транзакциями


В каждой категории есть свои операторы (перечислены не все операторы):

* DDL
    * CREATE - создание новой таблицы, СУБД, схемы
    * ALTER - изменение существующей таблицы, колонки
    * DROP - удаление существующих объектов из СУБД
* DML
    * SELECT - выбор данных
    * INSERT - добавление новых данных
    * UPDATE - обновление существующих данных
    * DELETE - удаление данных
* DCL
    * GRANT - предоставление пользователям разрешения на чтение/запись определенных объектов в СУБД
    * REVOKE - отзыв ранее предоставленных разрешений
* TCL
    * COMMIT - применение транзакции
    * ROLLBACK - откат всех изменений, сделанных в текущей транзакции

## SQL и Python

Для работы с реляционной СУБД в Python можно использовать два подхода:

* работать с библиотекой, которая соответствует конкретной СУБД, и использовать для работы с БД язык SQL. Например, для работы с SQLite используется модуль sqlite3
* работать с ORM, которая использует объектно-ориентированный подход для работы с БД. Например, SQLAlchemy

> **ORM (Object-Relational Mapping)** — техника программирования, предназначенная для отображения данных объектно-ориентированного языка программирования на хранилище с несовместимой системой типов, чаще всего на реляционную СУБД. В результате, программа видит вымышленную объектно-ориентированную СУБД, представленную ORM, и не задумывается о том, что стоит за ней. Это существенно облегчает программирование и делает код независящим от особенностей нижележащей СУБД.

# SQLite

SQLite — встраиваемая в процесс реализация SQL-машины. SQLite часто используется как встроенная СУБД в приложениях.

> Слово SQL-сервер здесь не используем, потому что как таковой сервер там не нужен — весь функционал, который встраивается в SQL-сервер, реализован внутри библиотеки (и, соответственно, внутри программы, которая её использует).

## SQLite CLI

В комплекте поставки SQLite идёт также утилита для работы с SQLite в командной строке. Утилита представлена в виде исполняемого файла sqlite3 (sqlite3.exe для Windows), и с ее помощью можно вручную выполнять команды SQL.

С помощью этой утилиты очень удобно проверять правильность команд SQL, а также в целом знакомиться с языком SQL.

Попробуем с помощью этой утилиты разобраться с базовыми командами SQL, которые понадобятся для работы с БД.

Для начала разберемся, как создавать БД. Для того, чтобы создать БД (или открыть уже созданную), надо просто вызвать sqlite3 таким образом:

```sql
$ sqlite3 testDB.db
SQLite version 3.8.7.1 2014-10-29 13:59:56
Enter ".help" for usage hints.
sqlite>
```

Внутри sqlite3 можно выполнять команды SQL или так называемые метакоманды (или dot-команды).

К метакомандам относятся несколько специальных команд для работы с SQLite. Они относятся только к утилите sqlite3, а не к SQL языку. В конце этих команд `;` ставить не нужно.

Примеры метакоманд:

* `.help` - подсказка со списком всех метакоманд
* `.exit` или `.quit` - выход из сессии sqlite3
* `.databases` - показывает присоединенные БД
* `.tables` - показывает доступные таблицы

Примеры выполнения:

```sql
sqlite> .help
.backup ?DB? FILE      Backup DB (default "main") to FILE
.bail ON|OFF           Stop after hitting an error.  Default OFF
.databases             List names and files of attached databases
...

sqlite> .databases
seq  name      file
---  --------  ----------------------------------
0    main      /home/user/db/testDB.db
```

# Основы SQL (в sqlite3 CLI)

## `CREATE`

Оператор `CREATE` позволяет создавать таблицы.

Сначала подключимся к базе данных или создадим ее с помощью litecli:

```sql
$ litecli new_db.db
Version: 1.0.0
Mail: https://groups.google.com/forum/#!forum/litecli-users
Github: https://github.com/dbcli/litecli
new_db.db>
```

Создадим таблицу switch, в которой хранится информация о коммутаторах:

```sql
new_db.db> create table switch (mac text not NULL primary key, hostname text, model text, location text);
```

В данном примере мы описали таблицу switch: определили, какие поля будут в таблице, и значения какого типа будут в них находиться.

Кроме того, поле mac является первичным ключом. Это автоматически значит, что:

* поле должно быть уникальным
* в нём не может находиться значение NULL (в SQLite это надо задавать явно)

В этом примере это вполне логично, так как MAC-адрес должен быть уникальным.

На данный момент записей в таблице нет, есть только ее определение. Просмотреть определение можно такой командой:

```sql
new_db.db> .schema switch
+-----------------------------------------------------------------------------------------------+
| sql                                                                                           |
+-----------------------------------------------------------------------------------------------+
| CREATE TABLE switch (mac text not NULL primary key, hostname text, model text, location text) |
+-----------------------------------------------------------------------------------------------+
```

## `DROP`

Оператор `DROP` удаляет таблицу вместе со схемой и всеми данными.

Удалить таблицу можно так:

```sql
new_db.db> DROP table switch;
You're about to run a destructive command.
Do you want to proceed? (y/n): y
```

## `INSERT`

Оператор `INSERT` используется для добавления данных в таблицу.

> Если таблица была удалена на предыдущем шаге, надо ее создать:
> 
> ```sql
> new_db.db> create table switch (mac text not NULL primary key, hostname text, model text, location text);
> ```

Есть несколько вариантов добавления записей, в зависимости от того, все ли поля будут заполнены, и будут ли они идти по порядку определения полей или нет.

Если указываются значения для всех полей, добавить запись можно таким образом (порядок полей должен соблюдаться):

```sql
new_db.db> INSERT into switch values ('0010.A1AA.C1CC', 'sw1', 'Cisco 3750', 'London, Green Str');
```

Если нужно указать не все поля или указать их в произвольном порядке, используется такая запись:

```sql
new_db.db> INSERT into switch (mac, model, location, hostname) values ('0020.A2AA.C2CC', 'Cisco 3850', 'London, Green Str', 'sw2');
```

## `SELECT`

Оператор `SELECT` позволяет запрашивать информацию в таблице.

Например:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str |
+----------------+----------+------------+-------------------+
```

`SELECT *` означает, что нужно вывести все поля таблицы. Следом указывается, из какой таблицы запрашиваются данные: `from switch`.

Таким образом можно указывать конкретные столбцы, которые нужно вывести и в каком порядке:

```sql
new_db.db> SELECT hostname, mac, model from switch;
+----------+----------------+------------+
| hostname | mac            | model      |
+----------+----------------+------------+
| sw1      | 0010.A1AA.C1CC | Cisco 3750 |
| sw2      | 0020.A2AA.C2CC | Cisco 3850 |
+----------+----------------+------------+
```

## `WHERE`

Оператор `WHERE` используется для уточнения запроса. С помощью этого оператора можно указывать определенные условия, по которым отбираются данные. Если условие выполнено, возвращается соответствующее значение из таблицы, если нет - не возвращается.

Сейчас в таблице switch всего две записи:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str |
+----------------+----------+------------+-------------------+
```

Чтобы в таблице было больше записей, надо создать еще несколько строк. В litecli есть команда source, которая позволяет загружать команды SQL из файла.

Для добавления записей заготовлен файл add_rows_to_testdb.txt:

```sql
INSERT into switch values ('0030.A3AA.C1CC', 'sw3', 'Cisco 3750', 'London, Green Str');
INSERT into switch values ('0040.A4AA.C2CC', 'sw4', 'Cisco 3850', 'London, Green Str');
INSERT into switch values ('0050.A5AA.C3CC', 'sw5', 'Cisco 3850', 'London, Green Str');
INSERT into switch values ('0060.A6AA.C4CC', 'sw6', 'C3750', 'London, Green Str');
INSERT into switch values ('0070.A7AA.C5CC', 'sw7', 'Cisco 3650', 'London, Green Str');
```

Для загрузки команд из файла надо выполнить команду:

```sql
new_db.db> source add_rows_to_testdb.txt
```

Теперь таблица switch выглядит так:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str |
+----------------+----------+------------+-------------------+
```

С помощью оператора `WHERE` можно показать только те коммутаторы, модель которых 3850:

```sql
new_db.db> SELECT * from switch WHERE model = 'Cisco 3850';
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str |
+----------------+----------+------------+-------------------+
```

Оператор `WHERE` позволяет указывать не только конкретное значение поля. Если добавить к нему оператор `LIKE`, можно указывать шаблон поля.

`LIKE` с помощью символов `_` и `% `указывает, на что должно быть похоже значение:

* `_` - обозначает один символ или число
* `%` - обозначает ноль, один или много символов

Например, если поле model записано в разном формате, с помощью предыдущего запроса не получится вывести нужные коммутаторы.

Например, у коммутатора sw6 поле model записано в таком формате: C3750, а у коммутаторов sw1 и sw3 в таком: Cisco 3750.

В таком варианте запрос с оператором `WHERE` не покажет sw6:

```sql
new_db.db> SELECT * from switch WHERE model = 'Cisco 3750';
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str |
+----------------+----------+------------+-------------------+
```

Если вместе с оператором `WHERE` использовать оператор `LIKE`:

```sql
new_db.db> SELECT * from switch WHERE model LIKE '%3750';
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str |
+----------------+----------+------------+-------------------+
```

## `ALTER`

Оператор `ALTER` позволяет менять существующую таблицу: добавлять новые колонки или переименовывать таблицу.

Добавим в таблицу новые поля:

* mngmt_ip - IP-адрес коммутатора в менеджмент VLAN
* mngmt_vid - VLAN ID (номер VLAN) для менеджмент VLAN

Добавление записей с помощью команды ALTER:

```sql
new_db.db> ALTER table switch ADD COLUMN mngmt_ip text;
You're about to run a destructive command.
Do you want to proceed? (y/n): y

new_db.db> ALTER table switch ADD COLUMN mngmt_vid integer;
You're about to run a destructive command.
Do you want to proceed? (y/n): y
```

Теперь таблица выглядит так (новые поля установлены в значение NULL):

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+----------+-----------+
| mac            | hostname | model      | location          | mngmt_ip | mngmt_vid |
+----------------+----------+------------+-------------------+----------+-----------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str | <null>   | <null>    |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | <null>   | <null>    |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str | <null>   | <null>    |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | <null>   | <null>    |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | <null>   | <null>    |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | <null>   | <null>    |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | <null>   | <null>    |
+----------------+----------+------------+-------------------+----------+-----------+
```

## `UPDATE`

Оператор `UPDATE` используется для изменения существующей записи таблицы.

Обычно, `UPDATE` используется вместе с оператором `WHERE`, чтобы уточнить, какую именно запись необходимо изменить.

С помощью `UPDATE` можно заполнить новые столбцы в таблице.

Например, добавить IP-адрес для коммутатора sw1:

```sql
new_db.db> UPDATE switch set mngmt_ip = '10.255.1.1' WHERE hostname = 'sw1';
```

Теперь таблица выглядит так:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str | 10.255.1.1 | <null>    |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str | <null>     | <null>    |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | <null>     | <null>    |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | <null>     | <null>    |
+----------------+----------+------------+-------------------+------------+-----------+
```

Аналогичным образом можно изменить и номер VLAN:

```sql
new_db.db> UPDATE switch set mngmt_vid = 255 WHERE hostname = 'sw1';


new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str | <null>     | <null>    |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | <null>     | <null>    |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | <null>     | <null>    |
+----------------+----------+------------+-------------------+------------+-----------+
```

Можно изменить несколько полей за раз:

```sql
new_db.db> UPDATE switch set mngmt_ip = '10.255.1.2', mngmt_vid = 255 WHERE hostname = 'sw2'

new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str | <null>     | <null>    |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | <null>     | <null>    |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | <null>     | <null>    |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | <null>     | <null>    |
+----------------+----------+------------+-------------------+------------+-----------+
```

Чтобы не заполнять поля mngmt_ip и mngmt_vid вручную, заполним остальное из файла update_fields_in_testdb.txt (команда `source update_fields_in_testdb.txt`):

```sql
UPDATE switch set mngmt_ip = '10.255.1.3', mngmt_vid = 255 WHERE hostname = 'sw3';
UPDATE switch set mngmt_ip = '10.255.1.4', mngmt_vid = 255 WHERE hostname = 'sw4';
UPDATE switch set mngmt_ip = '10.255.1.5', mngmt_vid = 255 WHERE hostname = 'sw5';
UPDATE switch set mngmt_ip = '10.255.1.6', mngmt_vid = 255 WHERE hostname = 'sw6';
UPDATE switch set mngmt_ip = '10.255.1.7', mngmt_vid = 255 WHERE hostname = 'sw7';
```

После загрузки команд таблица выглядит так:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.A1AA.C1CC | sw1      | Cisco 3750 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str | 10.255.1.3 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

Теперь предположим, что sw1 был заменен с модели 3750 на модель 3850. Соответственно, изменилось не только поле модель, но и поле MAC-адрес.

Внесение изменений:

```sql
new_db.db> UPDATE switch set model = 'Cisco 3850', mac = '0010.D1DD.E1EE' WHERE hostname = 'sw1';
```

Результат будет таким:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3750 | London, Green Str | 10.255.1.3 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

## `REPLACE`

Оператор `REPLACE` используется для добавления или замены данных в таблице.

> Оператор `REPLACE` может поддерживаться не во всех СУБД.

Когда возникает нарушение условия уникальности поля, выражение с оператором `REPLACE`:

* удаляет существующую строку, которая вызвала нарушение
* добавляет новую строку

Пример нарушения правила уникальности:

```sql
new_db.db> INSERT INTO switch VALUES ('0030.A3AA.C1CC', 'sw3', 'Cisco 3850', 'London, Green Str', '10.255.1.3', 255);
UNIQUE constraint failed: switch.mac
```

У выражения `REPLACE` есть два вида:

```sql
new_db.db> INSERT OR REPLACE INTO switch VALUES ('0030.A3AA.C1CC', 'sw3', 'Cisco 3850', 'London, Green Str', '10.255.1.3', 255);
```

Или более короткий вариант:

```sql
new_db.db> REPLACE INTO switch VALUES ('0030.A3AA.C1CC', 'sw3', 'Cisco 3850', 'London, Green Str', '10.255.1.3', 255);
```

Результатом любой из этих команд будет замена модели коммутатора sw3:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

В данном случае MAC-адрес в новой записи совпадает с уже существующей, поэтому происходит замена.

> Если были указаны не все поля, в новой записи будут только те поля, которые были указаны. Это связано с тем, что `REPLACE` сначала удаляет существующую запись.

При добавлении записи, для которой не возникает нарушения уникальности поля, `REPLACE` работает как обычный `INSERT`:

```sql
new_db.db> REPLACE INTO switch VALUES ('0080.A8AA.C8CC', 'sw8', 'Cisco 3850', 'London, Green Str', '10.255.1.8', 255);


new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
| 0080.A8AA.C8CC | sw8      | Cisco 3850 | London, Green Str | 10.255.1.8 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

## `DELETE`

Оператор `DELETE` используется для удаления записей. Как правило, он используется вместе с оператором `WHERE`.

Например, таблица switch выглядит так:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
| 0080.A8AA.C8CC | sw8      | Cisco 3850 | London, Green Str | 10.255.1.8 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

Удаление информации про коммутатор sw8 выполняется таким образом:

```sql
new_db.db> DELETE from switch where hostname = 'sw8';
You're about to run a destructive command.
Do you want to proceed? (y/n): y
```

Теперь в таблице нет строки с коммутатором sw8:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

## `ORDER BY`

Оператор `ORDER BY` используется для сортировки вывода по определенному полю, по возрастанию или убыванию. Для этого он добавляется к оператору `SELECT`.

Если выполнить простой запрос `SELECT`, вывод будет таким:

```sql
new_db.db> SELECT * from switch;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

С помощью оператора `ORDER BY` можно вывести записи в таблице switch, отсортировав их по имени коммутаторов:

```sql
new_db.db> SELECT * from switch ORDER BY hostname ASC;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

По умолчанию сортировка выполняется по возрастанию, поэтому в запросе можно было не указывать параметр `ASC`:

```sql
new_db.db> SELECT * from switch ORDER BY hostname;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

Сортировка по IP-адресу по убыванию:

```sql
SELECT * from switch ORDER BY mngmt_ip DESC;
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

## `AND`

Оператор `AND` позволяет группировать несколько условий:

```sql
new_db.db> select * from switch where model = 'Cisco 3850' and mngmt_ip LIKE '10.255.%';
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

## `OR`

Оператор `OR`:

```sql
new_db.db> select * from switch where model LIKE '%3750' or model LIKE '%3850';
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0060.A6AA.C4CC | sw6      | C3750      | London, Green Str | 10.255.1.6 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

## `IN`

Оператор `IN`:

```sql
new_db.db> select * from switch where model in ('Cisco 3750', 'C3750');
+----------------+----------+-------+-------------------+------------+-----------+
| mac            | hostname | model | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+-------+-------------------+------------+-----------+
| 0060.A6AA.C4CC | sw6      | C3750 | London, Green Str | 10.255.1.6 | 255       |
+----------------+----------+-------+-------------------+------------+-----------+
```

## `NOT`

Оператор `NOT`:

```sql
new_db.db> select * from switch where model not in ('Cisco 3750', 'C3750');
+----------------+----------+------------+-------------------+------------+-----------+
| mac            | hostname | model      | location          | mngmt_ip   | mngmt_vid |
+----------------+----------+------------+-------------------+------------+-----------+
| 0010.D1DD.E1EE | sw1      | Cisco 3850 | London, Green Str | 10.255.1.1 | 255       |
| 0020.A2AA.C2CC | sw2      | Cisco 3850 | London, Green Str | 10.255.1.2 | 255       |
| 0040.A4AA.C2CC | sw4      | Cisco 3850 | London, Green Str | 10.255.1.4 | 255       |
| 0050.A5AA.C3CC | sw5      | Cisco 3850 | London, Green Str | 10.255.1.5 | 255       |
| 0070.A7AA.C5CC | sw7      | Cisco 3650 | London, Green Str | 10.255.1.7 | 255       |
| 0030.A3AA.C1CC | sw3      | Cisco 3850 | London, Green Str | 10.255.1.3 | 255       |
+----------------+----------+------------+-------------------+------------+-----------+
```

# Модуль sqlite3

Для работы с SQLite в Python используется модуль sqlite3.

Объект **Connection** - это подключение к конкретной БД. Можно сказать, что этот объект представляет БД.

Пример создания подключения:

In [ ]:
import sqlite3

connection = sqlite3.connect('dhcp_snooping.db')

После создания соединения надо создать объект **Cursor** - это основной способ работы с БД.

Создается курсор из соединения с БД:

In [ ]:
import sqlite3

connection = sqlite3.connect('dhcp_snooping.db')
cursor = connection.cursor()

## Выполнение команд SQL

Для выполнения команд SQL в модуле есть несколько методов:

* `execute` - метод для выполнения одного выражения SQL
* `executemany` - метод позволяет выполнить одно выражение SQL для последовательности параметров (или для итератора)
* `executescript` - метод позволяет выполнить несколько выражений SQL за один раз

### Метод `execute`

Метод `execute` позволяет выполнить одну команду SQL.

In [ ]:
import sqlite3

# Сначала надо создать соединение и курсор:

connection = sqlite3.connect('sw_inventory.db')
cursor = connection.cursor()

# Создание таблицы switch с помощью метода execute:

cursor.execute("create table switch (mac text not NULL primary key, hostname text, model text, location text)")

'''
Выражения SQL могут быть параметризированы - вместо данных можно подставлять специальные значения. За счет этого можно использовать одну и ту же команду SQL для передачи разных данных.

Например, таблицу switch нужно заполнить данными из списка data:
'''

data = [
    ('0000.AAAA.CCCC', 'sw1', 'Cisco 3750', 'London, Green Str'),
    ('0000.BBBB.CCCC', 'sw2', 'Cisco 3780', 'London, Green Str'),
    ('0000.AAAA.DDDD', 'sw3', 'Cisco 2960', 'London, Green Str'),
    ('0011.AAAA.CCCC', 'sw4', 'Cisco 3750', 'London, Green Str')]

# Для этого можно использовать запрос вида:

query = "INSERT into switch values (?, ?, ?, ?)"

'''
Знаки вопроса в команде используются для подстановки данных, которые будут передаваться методу `execute`.

Теперь можно передать данные таким образом:
'''

for row in data:
    cursor.execute(query, row)

'''
Второй аргумент, который передается методу execute, должен быть кортежем. Если нужно передать кортеж с одним элементом, используется запись (value, ).

Чтобы изменения были применены, нужно выполнить commit (обратите внимание, что метод commit вызывается у соединения):
'''

connection.commit()

Теперь при запросе из командной строки sqlite3, можно увидеть эти строки в таблице switch:

```sql
$ litecli sw_inventory.db
Version: 1.0.0
Mail: https://groups.google.com/forum/#!forum/litecli-users
Github: https://github.com/dbcli/litecli
sw_inventory.db> SELECT * from switch;
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0000.AAAA.CCCC | sw1      | Cisco 3750 | London, Green Str |
| 0000.BBBB.CCCC | sw2      | Cisco 3780 | London, Green Str |
| 0000.AAAA.DDDD | sw3      | Cisco 2960 | London, Green Str |
| 0011.AAAA.CCCC | sw4      | Cisco 3750 | London, Green Str |
+----------------+----------+------------+-------------------+
```

## Метод `executemany`

Метод `executemany` позволяет выполнить одну команду SQL для последовательности параметров (или для итератора).

С помощью метода `executemany` в таблицу switch можно добавить аналогичный список данных одной командой.

Например, в таблицу switch надо добавить данные из списка data2:

```py
data2 = [
    ('0000.1111.0001', 'sw5', 'Cisco 3750', 'London, Green Str'),
    ('0000.1111.0002', 'sw6', 'Cisco 3750', 'London, Green Str'),
    ('0000.1111.0003', 'sw7', 'Cisco 3750', 'London, Green Str'),
    ('0000.1111.0004', 'sw8', 'Cisco 3750', 'London, Green Str')]
```

Для этого нужно использовать аналогичный запрос вида:

```py
query = "INSERT into switch values (?, ?, ?, ?)"
```

Теперь можно передать данные методу `executemany`:

```py
cursor.executemany(query, data2)
connection.commit()
```
После выполнения `commit` данные доступны в таблице:

```sql
$ litecli sw_inventory.db
Version: 1.0.0
Mail: https://groups.google.com/forum/#!forum/litecli-users
Github: https://github.com/dbcli/litecli
sw_inventory.db> SELECT * from switch;
+----------------+----------+------------+-------------------+
| mac            | hostname | model      | location          |
+----------------+----------+------------+-------------------+
| 0000.AAAA.CCCC | sw1      | Cisco 3750 | London, Green Str |
| 0000.BBBB.CCCC | sw2      | Cisco 3780 | London, Green Str |
| 0000.AAAA.DDDD | sw3      | Cisco 2960 | London, Green Str |
| 0011.AAAA.CCCC | sw4      | Cisco 3750 | London, Green Str |
| 0000.1111.0001 | sw5      | Cisco 3750 | London, Green Str |
| 0000.1111.0002 | sw6      | Cisco 3750 | London, Green Str |
| 0000.1111.0003 | sw7      | Cisco 3750 | London, Green Str |
| 0000.1111.0004 | sw8      | Cisco 3750 | London, Green Str |
+----------------+----------+------------+-------------------+
```

Метод `executemany` подставил соответствующие кортежи в команду SQL, и все данные добавились в таблицу.

### Метод `executescript`

Метод `executescript` позволяет выполнить несколько выражений SQL за один раз.

Особенно удобно использовать этот метод при создании таблиц:

In [ ]:
import sqlite3

connection = sqlite3.connect('new_db.db')
cursor = connection.cursor()

cursor.executescript('''
    create table switches(
        hostname     text not NULL primary key,
        location     text
    );
    
    create table dhcp(
        mac          text not NULL primary key,
        ip           text,
        vlan         text,
        interface    text,
        switch       text not null references switches(hostname)
    );
''')

## Получение результатов запроса

Для получения результатов запроса в sqlite3 есть несколько способов:

* использование методов `fetch` - в зависимости от метода возвращаются одна, несколько или все строки
* использование курсора как итератора - возвращается итератор

### Метод `fetchone`

Метод `fetchone` возвращает одну строку данных.

Пример получения информации из базы данных sw_inventory.db:

In [ ]:
import sqlite3
connection = sqlite3.connect('sw_inventory.db')
cursor = connection.cursor()
cursor.execute('select * from switch')
print(cursor.fetchone())
print(cursor.fetchone())

Обратите внимание, что хотя запрос SQL подразумевает, что запрашивалось всё содержимое таблицы, метод `fetchone` вернул только одну строку. И если повторно вызвать метод, он вернет следующую строку.

Аналогичным образом метод будет возвращать следующие строки. После обработки всех строк метод начинает возвращать None.

За счет этого метод можно использовать в цикле, например, так:

In [ ]:
import sqlite3
connection = sqlite3.connect('sw_inventory.db')
cursor = connection.cursor()

cursor.execute('select * from switch')

while True:
    next_row = cursor.fetchone()
    if next_row:
        print(next_row)
    else:
        break

### Метод `fetchmany`

Метод `fetchmany` возвращает список строк данных.

Синтаксис метода:

```py
cursor.fetchmany([size=cursor.arraysize])
```

С помощью параметра `size` можно указывать, какое количество строк возвращается. По умолчанию параметр `size` равен значению `cursor.arraysize`.

Например, таким образом можно возвращать по три строки из запроса:

In [ ]:
import sqlite3
connection = sqlite3.connect('sw_inventory.db')
cursor = connection.cursor()

cursor.execute('select * from switch')

from pprint import pprint

while True:
    three_rows = cursor.fetchmany(3)
    if three_rows:
        pprint(three_rows)
    else:
        break

Метод выдает нужное количество строк, а если строк осталось меньше, чем параметр `size`, то оставшиеся строки.

### Метод `fetchall`

Метод `fetchall` возвращает все строки в виде списка:

In [ ]:
import sqlite3
connection = sqlite3.connect('sw_inventory.db')
cursor = connection.cursor()

cursor.execute('select * from switch')

cursor.fetchall()

Важный аспект работы метода - он возвращает все оставшиеся строки.

То есть, если до метода `fetchall` использовался, например, метод `fetchone`, то метод `fetchall` вернет оставшиеся строки запроса. Метод fetchmany в этом аспекте работает аналогично.

## Cursor как итератор

Если нужно построчно обрабатывать результирующие строки, лучше использовать курсор как итератор. При этом не нужно использовать методы `fetch`.

При использовании методов `execute` возвращается курсор. А, так как курсор можно использовать как итератор, можно использовать его, например, в цикле `for`:

```py
In [34]: result = cursor.execute('select * from switch')

In [35]: for row in result:
    ...:     print(row)
    ...:
('0000.AAAA.CCCC', 'sw1', 'Cisco 3750', 'London, Green Str')
('0000.BBBB.CCCC', 'sw2', 'Cisco 3780', 'London, Green Str')
('0000.AAAA.DDDD', 'sw3', 'Cisco 2960', 'London, Green Str')
('0011.AAAA.CCCC', 'sw4', 'Cisco 3750', 'London, Green Str')
('0000.1111.0001', 'sw5', 'Cisco 3750', 'London, Green Str')
('0000.1111.0002', 'sw6', 'Cisco 3750', 'London, Green Str')
('0000.1111.0003', 'sw7', 'Cisco 3750', 'London, Green Str')
('0000.1111.0004', 'sw8', 'Cisco 3750', 'London, Green Str')
```

Аналогичный вариант отработает и без присваивания переменной:

```py
In [36]: for row in cursor.execute('select * from switch'):
    ...:     print(row)
    ...:
('0000.AAAA.CCCC', 'sw1', 'Cisco 3750', 'London, Green Str')
('0000.BBBB.CCCC', 'sw2', 'Cisco 3780', 'London, Green Str')
('0000.AAAA.DDDD', 'sw3', 'Cisco 2960', 'London, Green Str')
('0011.AAAA.CCCC', 'sw4', 'Cisco 3750', 'London, Green Str')
('0000.1111.0001', 'sw5', 'Cisco 3750', 'London, Green Str')
('0000.1111.0002', 'sw6', 'Cisco 3750', 'London, Green Str')
('0000.1111.0003', 'sw7', 'Cisco 3750', 'London, Green Str')
('0000.1111.0004', 'sw8', 'Cisco 3750', 'London, Green Str')
```

## Использование модуля sqlite3 без явного создания курсора

Методы `execute` доступны и в объекте Connection, и в объекте Cursor, а методы `fetch` доступны только в объекте Cursor.

При использовании методов `execute` с объектом Connection курсор возвращается как результат выполнения метода `execute`. Его можно использовать как итератор и получать данные без методов `fetch`. За счет этого при работе с модулем sqlite3 можно не создавать курсор.

Пример итогового скрипта:

In [ ]:
import sqlite3


data = [('0000.AAAA.CCCC', 'sw1', 'Cisco 3750', 'London, Green Str'),
        ('0000.BBBB.CCCC', 'sw2', 'Cisco 3780', 'London, Green Str'),
        ('0000.AAAA.DDDD', 'sw3', 'Cisco 2960', 'London, Green Str'),
        ('0011.AAAA.CCCC', 'sw4', 'Cisco 3750', 'London, Green Str')]


con = sqlite3.connect('sw_inventory2.db')

con.execute('''create table switch
            (mac text not NULL primary key, hostname text, model text, location text)'''
            )

query = 'INSERT into switch values (?, ?, ?, ?)'
con.executemany(query, data)
con.commit()

for row in con.execute('select * from switch'):
    print(row)

con.close()

## Обработка исключений

Посмотрим на пример использования метода `execute` при возникновении ошибки.

В таблице switch поле mac должно быть уникальным. И, если попытаться записать пересекающийся MAC-адрес, возникнет ошибка:

```py
In [37]: con = sqlite3.connect('sw_inventory2.db')

In [38]: query = "INSERT into switch values ('0000.AAAA.DDDD', 'sw7', 'Cisco 2960', 'London, Green Str')"

In [39]: con.execute(query)
------------------------------------------------------------
IntegrityError             Traceback (most recent call last)
<ipython-input-56-ad34d83a8a84> in <module>()
----> 1 con.execute(query)

IntegrityError: UNIQUE constraint failed: switch.mac
```

Соответственно, можно перехватить исключение:

```py
In [40]: try:
    ...:     con.execute(query)
    ...: except sqlite3.IntegrityError as e:
    ...:     print("Error occurred: ", e)
    ...:
Error occurred:  UNIQUE constraint failed: switch.mac
```

Обратите внимание, что надо перехватывать исключение `sqlite3.IntegrityError`, а не `IntegrityError`.